This work was influenced by some kernels. Help me to improve this kernel and to understand some graphs as i am new to this.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
raw_train=pd.read_csv('/kaggle/input/forest-cover-type-prediction/train.csv')
#Drop column ID
raw_train = raw_train.iloc[:,1:]
raw_train.head()

In [ ]:
raw_train.info()

In [ ]:
list(set(raw_train.dtypes.tolist()))

ll the features are in the form of Numeric only.

In [ ]:
raw_train.isnull().sum()

No attribute is missing as count is 15120 for all attributes. Hence, all rows can be used

In [ ]:
raw_train.shape

In [ ]:
raw_train.describe()

 Wilderness_Area and Soil_Type are one hot encoded. Hence, they could be converted back for some analysis
 
 Attributes Soil_Type7 and Soil_Type15 can be removed as they are constant
 
 Scales are not the same for all. Hence, rescaling and standardization may be necessary for some algos
 
 Horizontal_Distance_To_Hydrology, Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Horizontal_Distance_To_Fire_Points can have multiplier as their max value is much greater than the 75%. 		
 
 

In [ ]:
raw_train[raw_train.duplicated()]

There is no dupplicate records

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [ ]:
plt.figure(figsize=(9, 8))
sns.distplot(raw_train['Cover_Type'], color='g', hist_kws={'alpha': 0.4});

In [ ]:
raw_train.hist(figsize=(20, 30), bins=50, xlabelsize=8, ylabelsize=8); # ; avoid having the matplotlib verbose informations

In [ ]:
def plot_distribution(col):
    #plt.figure(figsize=(6,4))
    ax = sns.countplot(raw_train[col])
    height = sum([p.get_height() for p in ax.patches])
    for p in ax.patches:
            ax.annotate(f'{100*p.get_height()/height:.2f} %', (p.get_x()+0.3, p.get_height()+5000),animated=True)

In [ ]:
plot_distribution("Cover_Type")

We see that all classes have an equal presence. No class re-balancing is necessary

# Correlation Matrix

In [ ]:
plt.figure(figsize=(10,7))
size = 10
corr = raw_train.iloc[:,:size].corr()

#num_cols = raw_train.select_dtypes(exclude=['object']).columns  # Without this also, it will generate the same result by selecting only numeric columns
#corr = raw_train[num_cols].corr()
sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values,annot=True)

**A correlation matrix of the 10 numerical variables is created and plotted in Figure 4

There are six pairwise correlations that have a value higher than absolute 0.5

    HS.noon, HS.3pm (0.61)
    HD.9am, HS.3pm (-0.78)
    HD.Hyrdro, VD.Hyrdo (0.65)
    Slope, HS.noon (-0.61)
    Aspect, HS.9am (-0.59)
    Aspect, HS.3pm (0.64)
    Elevation, HD.Road (0.58)


In [ ]:
corr = raw_train.drop('Cover_Type', axis=1).corr() # We already examined SalePrice correlations
plt.figure(figsize=(20, 15))

sns.heatmap(corr[(corr >= 0.5) | (corr <= -0.4)], 
            cmap='viridis', vmax=1.0, vmin=-1.0, linewidths=0.1,
            annot=True, annot_kws={"size": 8}, square=True);

We can conclude that, by essence, some of those features may be combined between each other in order to reduce the number of features like HS.noon, HS.3pm (0.61)
,HD.9am, HS.3pm (-0.78)

In [ ]:


df_num_corr = raw_train.corr()
df_num_corr
#golden_features_list = df_num_corr[abs(df_num_corr) > 0.2].sort_values(ascending=False)
#print("There is {} strongly correlated values with Covet_Type:\n{}".format(len(golden_features_list), golden_features_list))



# Scatter Plot

In [ ]:
sns.pairplot(raw_train.iloc[:,:size])

The scatter plot reveals some interesting pairwise relationships.

    The hillshade at noon and 3pm creates an elipsoid.

    As the horizontal distance to a hydro increases, the variance in vertical distance increases.

    As slope increase a proportion othe data’s hillshade at noon decreases (probably due to the aspect.)

    H3.3pm has a sigmoid relationship with Aspect

    F. Similarily Aspect and HS.9am have a more difined sigmoid relationship.
    
    Can also use bokeh.chart module to draw scatter plot one two features which are correlated
    
    # Import the necessary modules
from bokeh.charts import Scatter, output_file, show

     #Construct the scatter plot
p = Scatter(iris, x='Petal_length', y='Petal_width', color="Class", title="Petal Length vs Petal Width",
            xlabel="Sepal Length", ylabel="Sepal Width")

     #Output the file 
output_file('scatter.html')

     #Show the scatter plot
show(p)


In [ ]:
for i in range(0, len(raw_train.columns), 5):
    sns.pairplot(data=raw_train,
                x_vars=raw_train.columns[i:i+5],
                y_vars=['Cover_Type'])

Elevation seems to be an important feature as there are variations wrt the Cover_Type

In [ ]:
raw_train.columns[0:5]

In [ ]:
bin_cols = [f'Wilderness_Area{i+1}' for i in range(4)]
print(bin_cols)

fig, ax = plt.subplots(1,6, figsize=(30,10))

for i, col in enumerate(bin_cols):
     ax0 = plt.subplot(1,4,i+1)
     
     #data[col].value_counts().plot.bar(color='pink')
     sns.countplot(f'{col}', data= raw_train)
     #print(ax0.patches)
     height = sum([p.get_height() for p in ax0.patches])
     for p in ax0.patches:
         #get_x : Return the left coord of the rectangle
         ax0.text(p.get_x()+p.get_width()/2., p.get_height(), f'{100*p.get_height()/height:.2f} %', ha='center') 
     plt.xlabel(f'{col}')
plt.suptitle('Distribution over binary feature of train data')

Wilderness_Area2 is very less

In [ ]:
# Plot wrt Cover_Type for binary features
fig, ax = plt.subplots(1,4, figsize=(30, 8))
for i in range(4): 
    sns.countplot(f'Wilderness_Area{i+1}', hue='Cover_Type', data=raw_train, ax=ax[i])
    ax[i].set_ylim([0, 3000])
    ax[i].set_title(f'Wilderness_Area{i+1}', fontsize=15)
fig.suptitle("Binary Feature Distribution Wildnerness (Train Data)", fontsize=20)
plt.show()

# Box plot of numeric data

In [ ]:
featuresToPlot=["Elevation","Aspect","Slope","Horizontal_Distance_To_Hydrology","Vertical_Distance_To_Hydrology",
                "Horizontal_Distance_To_Roadways","Horizontal_Distance_To_Fire_Points","Hillshade_9am",
                "Hillshade_Noon","Hillshade_3pm"]

fig, ax = plt.subplots(3,4 ,figsize=(30,10))
for i, col in enumerate(featuresToPlot):
    #print(i,col)
    plt.subplot(2,5,i+1)
    raw_train.boxplot( column =[col])
    

Find the outliers values using IQR

In [ ]:
#Need to work on Outliers
Q1=raw_train.quantile(0.25)
Q3=raw_train.quantile(0.75)
IQR=Q3-Q1
#print(IQR)
Q1['Elevation']

In [ ]:
#Find the outlier value for Vertical_Distance_To_Hydrology
print(len(raw_train[(raw_train['Vertical_Distance_To_Hydrology']<(Q1['Vertical_Distance_To_Hydrology']-1.5*IQR['Vertical_Distance_To_Hydrology']))]))
print(len(raw_train[(raw_train['Vertical_Distance_To_Hydrology']>(Q3['Vertical_Distance_To_Hydrology']+1.5*IQR['Vertical_Distance_To_Hydrology']))]))



#df = df[~((df < (Q1–1.5 * IQR)) |(df > (Q3 + 1.5 * IQR))).any(axis=1)]

There are total {7+579} outlier for Vartical_Distance_To_Hydrology. We can chack for all the records.

In [ ]:
df = raw_train[~((raw_train<(Q1-1.5 * IQR)) |(raw_train>(Q3+1.5*IQR))).all(axis=1)]
df.shape

In [ ]:
print((raw_train<(Q1-1.5 * IQR)) | (raw_train>(Q3+1.5*IQR)))

There are outliers in most of teh columns. We need work on this to remove the outliers.

In [ ]:
fig, ax = plt.subplots(2,5 ,figsize=(30,10))
for i, col in enumerate(featuresToPlot):
    #print(i,col)
    #plt.subplot(2,5,i+1)
    raw_train.boxplot(by='Cover_Type', column =[col])

# Categorical Data Exploration

In [ ]:
#Merge 4 category Wilderness Type into One column and check for the distribution
newLabels =["Rawah","Neota","ComanchePeak","CachePoudre"]
oldCols =["Wilderness_Area1","Wilderness_Area2","Wilderness_Area3","Wilderness_Area4"]
df_one_hot=raw_train[oldCols]
df_one_hot.head()
df_one_hot.idxmax(axis=1)
raw_train['Wild']=df_one_hot.idxmax(axis=1)
di = {"Wilderness_Area1":"Rawah","Wilderness_Area2":"Neota","Wilderness_Area3":"ComanchePeak","Wilderness_Area4":"CachePoudre"}
raw_train['Wild']=raw_train['Wild'].map(di)  



In [ ]:
sns.countplot(raw_train['Wild'])

In [ ]:
sns.countplot(f'Wild', hue='Cover_Type', data=raw_train)

# Feature Selection


## Removing features with low variance

In [ ]:
from sklearn.feature_selection import VarianceThreshold
sel_variance_threshold = VarianceThreshold() 
X_train_remove_variance = sel_variance_threshold.fit_transform(raw_train)
print(X_train_remove_variance.shape)

Variance is 0 for soil type  7 and 15 so they got removed usng this method. Actual data has 55 column and here we have 53 columns.

In [ ]:
raw_train.describe()

In [ ]:
X = raw_train.drop('Cover_Type',axis=1)
y = raw_train['Cover_Type']